In [4]:
import sys
sys.path.append('../')

import os
import time
import numpy as np

from torch.utils.data import DataLoader
from torch.autograd import Variable
from torch import autograd

import torch
import torch.nn as nn


from utils.tools import get_config, default_loader, is_image_file, normalize
from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import torchvision.utils as vutils

from sklearn.metrics import roc_auc_score
sys.path.append('../PerceptualSimilarity')
import models as PerceptualSimilarity

# personal library
from networks import autoencoder, simulator, discriminator
from dataloader import MVTecDataset

In [5]:
# 限制可以使用的 GPU
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [6]:
# HYPER parameters
num_epochs = 50000
batch_size = 32
val_batch_size = 4
ae_lr = 1e-4
s_lr = 5e-4
d_lr = 5e-4
weight_decay = 1e-5
UPSET=True
expName = 'AEGAN-exp(wood + L1)'
writer = SummaryWriter('checkpoints/'+expName)
TYPE='wood'

In [7]:
trainDatset = MVTecDataset.MVTecDataset(TYPE=TYPE, isTrain='train')
testDatset = MVTecDataset.MVTecDataset(TYPE=TYPE, isTrain='test')
valDataset = MVTecDataset.MVTecDataset(TYPE=TYPE, isTrain='val')

val_loader = DataLoader(
    dataset=valDataset,
    batch_size=val_batch_size, 
    shuffle=True,
    num_workers=4
)
train_loader = DataLoader(
    dataset=trainDatset,
    batch_size=batch_size, 
    shuffle=True,
    num_workers=4
)
test_loader = DataLoader(
    dataset=testDatset,
    batch_size=1,
    shuffle=True,
    num_workers=4
)

In [8]:
# Model
AE = autoencoder.Autoencoder().cuda()
S = nn.DataParallel(simulator.Simulator(3, 8)).cuda()
D = nn.DataParallel(discriminator.Discriminator(6, 16)).cuda()

# Loss
L1_loss = nn.L1Loss()
L2_loss = nn.MSELoss(reduction='none')
perceptual_loss = PerceptualSimilarity.PerceptualLoss(model='net-lin', net='alex', use_gpu=True, gpu_ids=[0])

# Optimizer
optimizer_AE = torch.optim.Adam(
    AE.parameters(), 
    lr=ae_lr,
    weight_decay=weight_decay
)
optimizer_S = torch.optim.Adam(
    S.parameters(), 
    lr=s_lr,
    weight_decay=weight_decay
)
optimizer_D = torch.optim.Adam(
    D.parameters(), 
    lr=d_lr,
    weight_decay=weight_decay
)


Tensor = torch.cuda.FloatTensor

Setting up Perceptual loss...
Loading model from: /root/AFS/Corn/AEGAN/PerceptualSimilarity/models/weights/v0.1/alex.pth
...[net-lin [alex]] initialized
...Done


In [9]:
AE.load_state_dict(torch.load('./save_weight/AE-wood-z-2x2-exp2/AE_2500.npy', map_location="cuda:0"), False)

AE = nn.DataParallel(AE)

In [10]:
# To Solve: RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED
torch.backends.cudnn.enabled = False 

# 拿掉煩人的 warning
import warnings
warnings.filterwarnings("ignore")

In [11]:
LAMBDA = 10

def calc_gradient_penalty(netD, real_data, fake_data):
    # print "real_data: ", real_data.size(), fake_data.size()
    BATCH_SIZE = real_data.size(0)
    alpha = torch.rand(BATCH_SIZE, 1)
    alpha = alpha.expand(BATCH_SIZE, real_data.nelement()//BATCH_SIZE).contiguous().view(BATCH_SIZE, 6, 256, 256)
    alpha = alpha.cuda()

    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = interpolates.cuda()
    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(
        outputs=disc_interpolates, 
        inputs=interpolates,
        grad_outputs=torch.ones(disc_interpolates.size()).cuda(),
        create_graph=True, 
        retain_graph=True, 
        only_inputs=True
    )[0]
    gradients = gradients.view(gradients.size(0), -1)

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty

def difNormalize(input_matrix, threshold=None):
    _min = torch.min(input_matrix)
    _max = torch.max(input_matrix)
    
    input_matrix = (input_matrix - _min) / (_max - _min)
    
    if threshold != None:
        input_matrix[input_matrix < threshold] = 0
        input_matrix[input_matrix >= threshold] = 1
        
    return input_matrix

In [ ]:
for epoch in range(num_epochs): 
    start = time.time()
    ######## GAN ################
    one = torch.FloatTensor([1])
    mone = one * -1
    
    one = one.cuda()
    mone = mone.cuda()
    
    one = one.mean()
    mone = mone.mean()
    ## ==== GAN --> D =====
    for i in range(3):
        for index, img in enumerate(train_loader):
            AE.eval(), S.train(), D.train()

            img = Variable(img).cuda()

            # ====== AE ======
            blur_image = AE(img)

            _bs, _c, _w, _h = blur_image.shape
            noise = torch.zeros(_bs, 1, _w, _h )
            noise = noise + (0.01**0.5)*torch.randn(_bs, 1, _w, _h)
            noise = noise.cuda()

            blur_image_with_noise = torch.cat([blur_image, noise], 1)
            fake_image = S(blur_image_with_noise) # 當成是 residual
            
            fake_image = fake_image + blur_image # blur image + residual
            
            fake_pair = torch.cat([img, fake_image], 1)
            real_pair = torch.cat([img, img[torch.randperm(img.size(0)), :, :, :]], 1) if UPSET else torch.cat([img, img], 1)
            # ====== Train D ======
            for p in D.parameters():
                p.requires_grad = True

            optimizer_AE.zero_grad()
            optimizer_S.zero_grad()
            optimizer_D.zero_grad()


            real_D = D(real_pair)
            real_D = real_D.mean()
            real_D.backward(mone)


            fake_D = D(fake_pair)
            fake_D = fake_D.mean()
            fake_D.backward(one)

            gradient_penalty = calc_gradient_penalty(D, real_pair, fake_pair)
            gradient_penalty.backward()

            cost_D = fake_D - real_D + gradient_penalty
            Wasserstein_D = real_D - fake_D
            optimizer_D.step()
    
    ## ==== GAN --> G =====
    for index, img in enumerate(train_loader):
        AE.eval(), S.train(), D.train()

        img = Variable(img).cuda()
        # ======AE======
        blur_image = AE(img)

        _bs, _c, _w, _h = blur_image.shape
        noise = torch.zeros(_bs, 1, _w, _h )
        noise = noise + (0.01**0.5)*torch.randn(_bs, 1, _w, _h)
        noise = noise.cuda()

        blur_image_with_noise = torch.cat([blur_image, noise], 1)
        fake_image = S(blur_image_with_noise)
        
        fake_image = fake_image + blur_image
        
        fake_pair = torch.cat([img, fake_image], 1)
        # ====== Train G ======
        for p in D.parameters():
            p.requires_grad = False
        
        G_L1 = L1_loss(img, fake_image)
        G = D(fake_pair)
        G = G.mean()
        G = G + G_L1 * ( G / G_L1)
        
        optimizer_AE.zero_grad()
        optimizer_S.zero_grad()
        optimizer_D.zero_grad()
        
        G.backward(mone)
        
        cost_G = -G
        optimizer_S.step()
        
    
    # validation set
    for index, val_img in enumerate(val_loader):
        AE.eval(), S.eval(), D.eval()

        val_img = Variable(val_img).cuda()
        # ======AE======
        val_blur_image = AE(val_img)

        _bs, _c, _w, _h = val_blur_image.shape
        noise = torch.zeros(_bs, 1, _w, _h )
        noise = noise + (0.01**0.5)*torch.randn(_bs, 1, _w, _h)
        noise = noise.cuda()

        val_blur_image_with_noise = torch.cat([val_blur_image, noise], 1)

        val_fake_image = S(val_blur_image_with_noise)       
        val_fake_image = val_fake_image + val_blur_image
        
        
        val_fake_pair = torch.cat([val_img, val_fake_image], 1)
        val_real_pair = torch.cat([val_img, val_img[torch.randperm(val_img.size(0)), :, :, :]], 1) if UPSET else torch.cat([val_img, val_img], 1)
        
        val_real_D = D(val_real_pair)
        val_real_D = val_real_D.mean()
        
        val_fake_D = D(val_fake_pair)
        val_fake_D = val_fake_D.mean()
        
        val_gradient_penalty = calc_gradient_penalty(D, val_real_pair, val_fake_pair)
        
        val_G_L1 = L1_loss(val_img, val_fake_image)
        # =========== Losses =========
        val_Wasserstein_D = val_real_D - val_fake_D
        
        val_cost_G = -val_fake_D
        val_cost_D = val_fake_D - val_real_D + val_gradient_penalty
    
    # evaluate
    test_total_AUC = 0
    test_total_AUC2 = 0
    test_total_image = 0

    for index, (test_img, mask) in enumerate(test_loader):
        AE.eval(), S.eval(), D.eval()
        test_img = Variable(test_img).cuda()
        test_blur_image = AE(test_img)

        _bs, _c, _w, _h = test_blur_image.shape
        noise = torch.zeros(_bs, 1, _w, _h )
        noise = noise + (0.01**0.5)*torch.randn(_bs, 1, _w, _h)
        noise = noise.cuda()

        test_blur_image_with_noise = torch.cat([test_blur_image, noise], 1)

        test_fake_image = S(test_blur_image_with_noise)       
        test_fake_image = test_fake_image + test_blur_image

        # 計算 dif (相似度以及 L2)
        dif, _ = perceptual_loss.forward(test_fake_image, test_img)
        l2Dif = L2_loss(test_fake_image, test_img)
        l2Dif = torch.mean(l2Dif, 1, True)
        
        pred_mask2 = difNormalize(dif)
        pred_mask2 = torch.flatten(pred_mask2[0])
        
        pred_mask = difNormalize(dif[0] * l2Dif[0])
        pred_mask = torch.flatten(pred_mask)
        
        mask = torch.mean(mask, 1, True)
        true_mask = mask[0].cpu().detach().numpy().flatten()
        true_mask = true_mask.astype(int)

        AUC = roc_auc_score(true_mask, pred_mask.cpu().detach().numpy())
        AUC2 = roc_auc_score(true_mask, pred_mask2.cpu().detach().numpy())

        test_total_AUC += AUC
        test_total_AUC2 += AUC2
        test_total_image += 1
    
    # =================== GAN log========================
    end = time.time()
    print('epoch [{}/{}] s_loss:{:.4f} d_loss:{:.4f} val_s_loss:{:.4f} val_d_loss:{:.4f} cost:{:.2f}'.format(epoch+1, num_epochs, cost_G.item(), cost_D.item(), val_cost_G.item(), val_cost_D.item(), end-start ))
    writer.add_scalars('eval', {
        "auc_roc_score": test_total_AUC / test_total_image,
        "auc_roc_score(w/o L2)": test_total_AUC2 / test_total_image,
    }, epoch)
    
    writer.add_scalars('loss', {
        "Wasserstein Distance": Wasserstein_D.item(),
        "Val Wasserstein Distance": val_Wasserstein_D.item(),
        "gradient penalty": gradient_penalty,
        "val gradient penalty": val_gradient_penalty
    }, epoch)
    
    writer.add_scalars('gan loss', {
        "l1_loss": G_L1.item(),
        "g_loss": cost_G.item(),
        "d_loss": cost_D.item(),
        "val_l1_loss": val_G_L1.item(),
        "val_g_loss": val_cost_G.item(),
        "val_d_loss": val_cost_D.item()
    }, epoch)

    writer.add_images('Blur', blur_image, epoch)
    writer.add_images('Reconstruct', fake_image, epoch)
    writer.add_images('Origin', img, epoch)

    writer.add_images('Val Blur', val_blur_image, epoch)
    writer.add_images('Val Reconstruct', val_fake_image, epoch)
    writer.add_images('Val Origin', val_img, epoch)


        
    if epoch % 10 == 0:
        if not os.path.exists('./save_weight/{}'.format(expName)):
            os.makedirs('./save_weight/{}'.format(expName))
        torch.save(S.state_dict(), './save_weight/{}/S_{}.npy'.format(expName, epoch))
        torch.save(D.state_dict(), './save_weight/{}/D_{}.npy'.format(expName, epoch))

epoch [1/50000] s_loss:1883.6453 d_loss:-219.3843 val_s_loss:507.6328 val_d_loss:418.7781 cost:25.63
epoch [2/50000] s_loss:1597.4044 d_loss:-225.0609 val_s_loss:877.4207 val_d_loss:-70.1391 cost:21.18
epoch [3/50000] s_loss:2224.2908 d_loss:-198.5246 val_s_loss:1143.3422 val_d_loss:-145.5718 cost:21.81
epoch [4/50000] s_loss:1453.7805 d_loss:-147.2471 val_s_loss:726.8510 val_d_loss:-48.4474 cost:22.60
epoch [5/50000] s_loss:1204.1757 d_loss:-115.5312 val_s_loss:612.3178 val_d_loss:-87.9609 cost:21.96
epoch [6/50000] s_loss:856.0828 d_loss:-105.1086 val_s_loss:401.2988 val_d_loss:-46.0550 cost:21.99
epoch [7/50000] s_loss:700.2130 d_loss:-116.1161 val_s_loss:432.7015 val_d_loss:-157.3034 cost:21.50
epoch [8/50000] s_loss:613.8437 d_loss:-133.3876 val_s_loss:944.3091 val_d_loss:-567.8423 cost:24.36
epoch [9/50000] s_loss:486.1114 d_loss:-94.5679 val_s_loss:388.3938 val_d_loss:-197.6541 cost:24.84
epoch [10/50000] s_loss:438.2088 d_loss:-122.0010 val_s_loss:195.7018 val_d_loss:73.6540 co

In [ ]:
AE.eval()
S.eval()
for index, img in enumerate(test_loader):
    test_img = Variable(img[0]).cuda()

    # ======AE======
    blur_image = AE(test_img)
    
    noise = torch.zeros(blur_image.shape[0], 1, blur_image.shape[2], blur_image.shape[3] )
    noise = noise + (0.01**0.5)*torch.randn(blur_image.shape[0], 1, blur_image.shape[2], blur_image.shape[3])
    noise = noise.cuda()
    blur_image_with_noise = torch.cat([blur_image, noise], 1)
    fake_image = S(blur_image_with_noise)
    
    
    vutils.save_image(fake_image[0], './test_result/{}_simulated.png'.format(index))
    vutils.save_image(blur_image[0], './test_result/{}_blur.png'.format(index))
    vutils.save_image(test_img, './test_result/{}_origin.png'.format(index))